# Set up your annotation team
Depending on the nature of your project and the size of your annotation team, you may want to have control over annotation overlap i.e., having multiple annotations for a single record. You will need to decide on this before pushing your dataset to Argilla, as this has implications on how your dataset is set up. Let's explore a few overlapping options.

## Full overlap
The Feedback Task supports having multiple annotations for your records. This means that all users with access to the dataset can give responses to all the records in the dataset. To have this full overlap just push the dataset (as detailed in [Create your dataset](guides/llms/practical_guides/create_and_import_dataset.html) in a workspace where all team members have access. Learn more about managing user access to workspaces [here](../../../getting_started/installation/configurations/user_management.md#creating-an-annotator-user-assigned-to-a-workspace).

## Zero overlap
If you only want one annotation per record, we recommend that you split your records into chunks and assign these to a single annotator. Then, you can create several datasets, one in each annotator's personal workspace, and add the records assigned to each of them.

In [ ]:

import httpx
import random
from collections import defaultdict

# make a request using your Argilla Client
rg_client= rg.active_client().client
auth_headers = {"X-Argilla-API-Key": rg_client.token}
http=httpx.Client(base_url=rg_client.base_url, headers=auth_headers)
users = http.get("/api/users").json()

# optional: filter users to get only those with annotator role
users = [u for u in users if u['role']=='annotator']

# optional: shuffle the records to get a random assignment
random.shuffle(records)

# build a dictionary where the key is the username and the value is the list of records assigned to them
assignments = defaultdict(list)

# divide your records in chunks of the same length as the users list and make the assignments
# you will need to follow the instructions to create and push a dataset for each of the key-value pairs in this dictionary
n = len(users)
chunked_records = [records[i:i + n] for i in range(0, len(records), n)]
for chunk in chunked_records:
    for idx, record in enumerate(chunk):
        assignments[users[idx]['username']].append(record)

## Controlled overlap
This option is optimal when you want to have annotation overlap, but up to a certain number and not with the whole team. This can be because you want your team to be more efficient or perhaps to calculate the agreement between pairs of annotators. In this case, you also need to create several datasets and push them to the annotators' personal workspaces with the difference that each record will appear in multiple datasets. 

Here's an example of how you can do those assignments:

In [ ]:
# code to assign with overlap
def assign_records(users, records, overlap):
    assignments = {user['username']: [] for user in users}
    random.shuffle(records)
    
    num_users = len(users)
    num_records = len(records)
    num_assignments = num_records * overlap
    
    assignments_per_user = num_assignments // num_users
    
    for i in range(num_records):
        record = records[i]
        
        for j in range(overlap):
            user_index = (i * overlap + j) % num_users
            user = users[user_index]['username']
            assignments[user].append(record)
    
    return assignments

assignments = assign_records(users, records, overlap=3)